In [1]:
from utils.processing import *
import numpy as np
import yaml
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# First Trials

In [2]:
with open("config/config.yaml", 'r') as ymlfile:
    config = yaml.safe_load(ymlfile)
    
LABEL_NAMES = config['LABEL_NAMES'][0]

In [3]:
dev_df, _ = load_file("data/development.csv", sep=",", dtype = {"id": str, "page_rank": int, "label": int})
dev_df_original = dev_df.copy()

  Development set: 79,997 samples, 7 features


# Build pipeline

In [4]:
from sklearn.model_selection import train_test_split


df_clean = preprocessing_data(dev_df_original)

X = df_clean.drop('label', axis=1)
y = df_clean['label']



X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y, shuffle=True)

Converting timestamp to datetime...
Dropping duplicates based on source, title, article, label keeping the most recent one...
Dropping duplicates that have the same source, title, article but different label...
Dropping id column...
  Preprocessed Data: 75,658 samples, 6 features


In [5]:
tfidf_args = {
    'text_cols': ['title', 'text'],
    'max_features': 5000,
    'ngram_range': (1, 2),
    'min_df': 5,
    'max_df': 0.7,
    'token_pattern': r'(?u)\b[a-zA-Z]{3,}\b',
    'stop_words': 'english'
}

preproc_pipe = pipeline(tfidf_args)

preproc_pipe.fit_transform(X_train, y_train)
feature_names = preproc_pipe.named_steps['processor'].get_feature_names_out()

X_train_processed = preproc_pipe.fit_transform(X_train, y_train)


X_val_processed = preproc_pipe.transform(X_val)

/Users/mottad/Library/CloudStorage/OneDrive-LuxotticaGroupS.p.A/Desktop/news_classification /ds_venv/lib/python3.11/site-packages/pandas/core/arrays/categorical.py:1752: RuntimeWarning: invalid value encountered in cast
  return np.asarray(ret, dtype=dtype)
/Users/mottad/Library/CloudStorage/OneDrive-LuxotticaGroupS.p.A/Desktop/news_classification /ds_venv/lib/python3.11/site-packages/pandas/core/arrays/categorical.py:1752: RuntimeWarning: invalid value encountered in cast
  return np.asarray(ret, dtype=dtype)
/Users/mottad/Library/CloudStorage/OneDrive-LuxotticaGroupS.p.A/Desktop/news_classification /ds_venv/lib/python3.11/site-packages/pandas/core/arrays/categorical.py:1752: RuntimeWarning: invalid value encountered in cast
  return np.asarray(ret, dtype=dtype)
/Users/mottad/Library/CloudStorage/OneDrive-LuxotticaGroupS.p.A/Desktop/news_classification /ds_venv/lib/python3.11/site-packages/pandas/core/arrays/categorical.py:1752: RuntimeWarning: invalid value encountered in cast
  retu

In [6]:
X_train_processed

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 3889175 stored elements and shape (60526, 5036)>

In [8]:
X_train_df = pd.DataFrame(
    X_train_processed.toarray(), # Convertiamo in denso solo per la visualizzazione
    columns=feature_names,
    index=X_train.index
)

X_train_df.head()

,text_pipeline__000,text_pipeline__000 jobs,text_pipeline__10,text_pipeline__10 000,text_pipeline__10 percent,text_pipeline__100,text_pipeline__11,text_pipeline__12,text_pipeline__13,text_pipeline__14,...,timestamp_transformer__month_sin,timestamp_transformer__month_cos,timestamp_transformer__week_sin,timestamp_transformer__week_cos,scale_num__page_rank,scale_num__n_links,scale_num__n_images,scale_num__n_ads,scale_num__n_feeds,scale_num__article_length
69842,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.224647e-16,-1.000000,-0.239316,-0.970942,-4.651192,0.796584,6.003724,-0.122096,-0.165952,0.045236
52331,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.224647e-16,-1.000000,-0.120537,-0.992709,0.275879,-0.288035,-0.331432,-0.122096,-0.165952,0.045236
11157,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.000000e-01,0.866025,0.822984,0.568065,0.275879,-0.288035,-0.331432,-0.122096,-0.165952,0.045236
8112,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-8.660254e-01,-0.500000,-0.885456,-0.464723,0.275879,0.796584,2.836146,-0.122096,-0.165952,0.045236
29337,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-8.660254e-01,-0.500000,-0.935016,-0.354605,0.275879,-0.288035,-0.331432,-0.122096,-0.165952,0.045236


In [11]:
# Recuperiamo il TF-IDF dalla pipeline
tfidf = preproc_pipe.named_steps['processor'].named_transformers_['text_pipeline'].named_steps['tfidf_vectorizer']

# Otteniamo i nomi delle parole
words = tfidf.get_feature_names_out()

# Vediamo le prime 50 parole in ordine alfabetico
print(words[:50])

# TRUCCO: Vediamo le parole con il punteggio IDF più basso (le più comuni)
import pandas as pd
idf_df = pd.DataFrame({'word': words, 'idf': tfidf.idf_}).sort_values(by='idf')
print(idf_df.head(20))

['000' '000 jobs' '10' '10 000' '10 percent' '100' '11' '12' '13' '14'
 '15' '150' '16' '17' '18' '19' '20' '200' '2000' '2001' '2002' '2003'
 '2004' '2005' '2006' '2007' '2008' '21' '22' '23' '24' '25' '26' '27'
 '28' '29' '2nd' '30' '300' '31' '32' '33' '34' '35' '36' '37' '38' '3g'
 '40' '400']
         word       idf
4158      the  1.278653
4382       to  1.552984
2843       of  1.630316
1997       in  1.659184
261       and  1.882495
2905       on  1.948262
1575      for  2.154606
4141     that  2.659526
4899     with  2.713135
3502  reuters  2.765523
2165       is  2.785911
403        at  2.809949
3586     said  2.837895
375        as  2.849800
700        by  2.866965
2054   in the  2.921994
2765      new  2.945877
2866   of the  2.969397
245        an  3.022024
1648     from  3.029415


In [7]:
import sys
sys.exit()

SystemExit: 

/Users/mottad/Library/CloudStorage/OneDrive-LuxotticaGroupS.p.A/Desktop/news_classification /ds_venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3709: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [13]:
X_train

,source,title,article,page_rank,timestamp,first_link_domain,title_suffix,article_length
69621,Wired,Vista Not Open to All,citing security concerns microsoft prevents ma...,5,2007-02-22 18:54:14,NaN,NaN,1
52389,New,Angry Putin rejects public Beslan inquiry,the russian president vladimir putin refused o...,5,NaT,NaN,NaN,1
11534,RedNova,BioPharm Informatics Announces New Name -- Lab...,biopharm informatics llc a premier provider of...,4,2007-02-20 17:50:13,NaN,SM,1
8278,Wired,Powell Concerned by Possible N.Korean Missile ...,secretary of state colin powell said on thursd...,5,2004-09-24 05:52:07,NaN,NaN,1
28853,Boston,Business notebook,2d drop shares of cambridge biotechnology comp...,4,2007-06-14 11:47:28,NaN,NaN,1
...,...,...,...,...,...,...,...,...
65103,Guardian,Russia puts price on Chechen rebels,video image of hostages sitting below explosiv...,5,NaT,NaN,NaN,1
66723,Yahoo,New intelligence reform bill may pit US Congre...,afp the powerful chairman of the us senate int...,5,NaT,NaN,AFP,1
26440,Boston,Britain says Iran is risking UN action,london iran risks action by the united nations...,5,2004-09-09 18:33:40,NaN,NaN,1
13073,Yahoo,Jury Selection Begins in Jackson Case (AP),ap michael jackson stood and smiled as he face...,5,NaT,NaN,AP,1


In [14]:
DatesEncoder = CyclicDateTransformer('timestamp')

X_train_new = DatesEncoder.fit_transform(X_train_new)

In [15]:
X_train_new[X_train_new['title'].str.startswith('Prescription drug ads simply not necessary')]

,title,article,page_rank,article_length,source_prob_cl_0,first_link_domain_prob_cl_0,title_suffix_prob_cl_0,source_prob_cl_1,first_link_domain_prob_cl_1,title_suffix_prob_cl_1,...,title_suffix_prob_cl_6,is_timestamp_missing,hour_sin,hour_cos,day_sin,day_cos,month_sin,month_cos,week_sin,week_cos


In [16]:
X_train_new = X_train_new.drop(columns = ['title', 'article'])

In [17]:
X_train_new

,page_rank,article_length,source_prob_cl_0,first_link_domain_prob_cl_0,title_suffix_prob_cl_0,source_prob_cl_1,first_link_domain_prob_cl_1,title_suffix_prob_cl_1,source_prob_cl_2,first_link_domain_prob_cl_2,...,title_suffix_prob_cl_6,is_timestamp_missing,hour_sin,hour_cos,day_sin,day_cos,month_sin,month_cos,week_sin,week_cos
69621,5,1,0.297824,0.297823,0.297822,0.135101,0.135099,0.135098,0.142683,0.142683,...,0.037835,0,-1.000000,-1.836970e-16,0.433884,-0.900969,0.500000,8.660254e-01,0.748511,6.631227e-01
52389,5,1,0.297824,0.148914,0.148912,0.135098,0.067548,0.067551,0.142684,0.571333,...,0.018917,1,0.500000,8.660254e-01,0.433884,-0.900969,0.866025,5.000000e-01,0.822984,5.680647e-01
11534,4,1,0.297822,0.432609,0.297825,0.135100,0.045033,0.135098,0.142681,0.380892,...,0.037835,0,-0.965926,-2.588190e-01,0.781831,0.623490,0.500000,8.660254e-01,0.748511,6.631227e-01
8278,5,1,0.148911,0.324456,0.432604,0.067550,0.033774,0.045033,0.571340,0.535668,...,0.012612,0,0.965926,2.588190e-01,-0.433884,-0.900969,-0.866025,-5.000000e-01,-0.992709,-1.205367e-01
28853,4,1,0.297819,0.459570,0.574453,0.135099,0.027020,0.033775,0.142683,0.428543,...,0.009459,0,0.258819,-9.659258e-01,0.433884,-0.900969,0.500000,-8.660254e-01,0.354605,-9.350162e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65103,5,1,0.089551,0.285094,0.278123,0.039663,0.137933,0.140825,0.009099,0.133604,...,0.037675,1,0.500000,8.660254e-01,0.433884,-0.900969,0.866025,5.000000e-01,0.822984,5.680647e-01
66723,5,1,0.530488,0.285105,0.907613,0.060874,0.137929,0.032043,0.113650,0.133602,...,0.013903,1,0.500000,8.660254e-01,0.433884,-0.900969,0.866025,5.000000e-01,0.822984,5.680647e-01
26440,5,1,0.243512,0.285116,0.278141,0.253974,0.137924,0.140821,0.007153,0.133599,...,0.037674,0,-1.000000,-1.836970e-16,0.433884,-0.900969,-0.866025,-5.000000e-01,-0.935016,-3.546049e-01
13073,5,1,0.530527,0.285133,0.529096,0.060868,0.137921,0.009864,0.113641,0.133595,...,0.010712,1,0.500000,8.660254e-01,0.433884,-0.900969,0.866025,5.000000e-01,0.822984,5.680647e-01


In [34]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression

def train_and_evaluate(df):
    # Separiamo X e y
    X = df.drop(columns=['label'])
    y = df['label']

    # Identifichiamo le colonne
    prob_cols = [c for c in X.columns if '_prob_' in c]
    ordinal_cols = ['page_rank', 'article_length']
    binary_cols = ['is_timestamp_missing']

    # 1. PREPROCESSING
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), prob_cols),
            ('ord', MinMaxScaler(), ordinal_cols),
            ('pass', 'passthrough', binary_cols)
        ])

    # 2. DEFINIZIONE MODELLI
    models = {
        'LogisticRegression': LogisticRegression(class_weight='balanced'),
        'RandomForest': RandomForestClassifier(n_estimators=200, class_weight='balanced', random_state=42),
        'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
    }

    # 3. CROSS-VALIDATION (Macro F1)
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    results = {}
    print("Inizio Cross-Validation (Macro F1 Score):")
    for name, model in models.items():
        clf_pipeline = Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('classifier', model)
        ])
        
        scores = cross_val_score(clf_pipeline, X, y, cv=skf, scoring='f1_macro')
        results[name] = scores.mean()
        print(f"   {name}: {scores.mean():.4f} (+/- {scores.std() * 2:.4f})")

    return results

# Esecuzione
results = train_and_evaluate(pd.concat([X_train_new, y_train], axis=1))

Inizio Cross-Validation (Macro F1 Score):


/Users/mottad/Library/CloudStorage/OneDrive-LuxotticaGroupS.p.A/Desktop/news_classification /ds_venv/lib/python3.11/site-packages/pandas/core/arrays/categorical.py:1752: RuntimeWarning: invalid value encountered in cast
  return np.asarray(ret, dtype=dtype)
/Users/mottad/Library/CloudStorage/OneDrive-LuxotticaGroupS.p.A/Desktop/news_classification /ds_venv/lib/python3.11/site-packages/pandas/core/arrays/categorical.py:1752: RuntimeWarning: invalid value encountered in cast
  return np.asarray(ret, dtype=dtype)
/Users/mottad/Library/CloudStorage/OneDrive-LuxotticaGroupS.p.A/Desktop/news_classification /ds_venv/lib/python3.11/site-packages/pandas/core/arrays/categorical.py:1752: RuntimeWarning: invalid value encountered in cast
  return np.asarray(ret, dtype=dtype)
/Users/mottad/Library/CloudStorage/OneDrive-LuxotticaGroupS.p.A/Desktop/news_classification /ds_venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:406: ConvergenceWarning: lbfgs failed to converge after 100 it

   LogisticRegression: 0.4110 (+/- 0.0069)


/Users/mottad/Library/CloudStorage/OneDrive-LuxotticaGroupS.p.A/Desktop/news_classification /ds_venv/lib/python3.11/site-packages/pandas/core/arrays/categorical.py:1752: RuntimeWarning: invalid value encountered in cast
  return np.asarray(ret, dtype=dtype)
/Users/mottad/Library/CloudStorage/OneDrive-LuxotticaGroupS.p.A/Desktop/news_classification /ds_venv/lib/python3.11/site-packages/pandas/core/arrays/categorical.py:1752: RuntimeWarning: invalid value encountered in cast
  return np.asarray(ret, dtype=dtype)
/Users/mottad/Library/CloudStorage/OneDrive-LuxotticaGroupS.p.A/Desktop/news_classification /ds_venv/lib/python3.11/site-packages/pandas/core/arrays/categorical.py:1752: RuntimeWarning: invalid value encountered in cast
  return np.asarray(ret, dtype=dtype)
/Users/mottad/Library/CloudStorage/OneDrive-LuxotticaGroupS.p.A/Desktop/news_classification /ds_venv/lib/python3.11/site-packages/pandas/core/arrays/categorical.py:1752: RuntimeWarning: invalid value encountered in cast
  retu

   RandomForest: 0.4116 (+/- 0.0110)


/Users/mottad/Library/CloudStorage/OneDrive-LuxotticaGroupS.p.A/Desktop/news_classification /ds_venv/lib/python3.11/site-packages/pandas/core/arrays/categorical.py:1752: RuntimeWarning: invalid value encountered in cast
  return np.asarray(ret, dtype=dtype)
/Users/mottad/Library/CloudStorage/OneDrive-LuxotticaGroupS.p.A/Desktop/news_classification /ds_venv/lib/python3.11/site-packages/pandas/core/arrays/categorical.py:1752: RuntimeWarning: invalid value encountered in cast
  return np.asarray(ret, dtype=dtype)
/Users/mottad/Library/CloudStorage/OneDrive-LuxotticaGroupS.p.A/Desktop/news_classification /ds_venv/lib/python3.11/site-packages/pandas/core/arrays/categorical.py:1752: RuntimeWarning: invalid value encountered in cast
  return np.asarray(ret, dtype=dtype)
/Users/mottad/Library/CloudStorage/OneDrive-LuxotticaGroupS.p.A/Desktop/news_classification /ds_venv/lib/python3.11/site-packages/xgboost/training.py:199: UserWarning: [13:00:23] WARNING: /Users/runner/work/xgboost/xgboost/src

   XGBoost: 0.4246 (+/- 0.0084)


/Users/mottad/Library/CloudStorage/OneDrive-LuxotticaGroupS.p.A/Desktop/news_classification /ds_venv/lib/python3.11/site-packages/pandas/core/arrays/categorical.py:1752: RuntimeWarning: invalid value encountered in cast
  return np.asarray(ret, dtype=dtype)
/Users/mottad/Library/CloudStorage/OneDrive-LuxotticaGroupS.p.A/Desktop/news_classification /ds_venv/lib/python3.11/site-packages/pandas/core/arrays/categorical.py:1752: RuntimeWarning: invalid value encountered in cast
  return np.asarray(ret, dtype=dtype)


In [35]:
results

{'LogisticRegression': np.float64(0.41096886276326383),
 'RandomForest': np.float64(0.4115671921399812),
 'XGBoost': np.float64(0.424577147080066)}

69621    2
52389    0
11534    2
8278     0
28853    1
        ..
65103    0
66723    0
26440    0
13073    3
24804    2
Name: label, Length: 60526, dtype: int64

In [29]:
pd.concat([X_train_new, y_train], axis=1)

,page_rank,article_length,source_prob_cl_0,first_link_domain_prob_cl_0,title_suffix_prob_cl_0,source_prob_cl_1,first_link_domain_prob_cl_1,title_suffix_prob_cl_1,source_prob_cl_2,first_link_domain_prob_cl_2,...,is_timestamp_missing,hour_sin,hour_cos,day_sin,day_cos,month_sin,month_cos,week_sin,week_cos,label
69621,5,1,0.297824,0.297823,0.297822,0.135101,0.135099,0.135098,0.142683,0.142683,...,0,-1.000000,-1.836970e-16,0.433884,-0.900969,0.500000,8.660254e-01,0.748511,6.631227e-01,2
52389,5,1,0.297824,0.148914,0.148912,0.135098,0.067548,0.067551,0.142684,0.571333,...,1,0.500000,8.660254e-01,0.433884,-0.900969,0.866025,5.000000e-01,0.822984,5.680647e-01,0
11534,4,1,0.297822,0.432609,0.297825,0.135100,0.045033,0.135098,0.142681,0.380892,...,0,-0.965926,-2.588190e-01,0.781831,0.623490,0.500000,8.660254e-01,0.748511,6.631227e-01,2
8278,5,1,0.148911,0.324456,0.432604,0.067550,0.033774,0.045033,0.571340,0.535668,...,0,0.965926,2.588190e-01,-0.433884,-0.900969,-0.866025,-5.000000e-01,-0.992709,-1.205367e-01,0
28853,4,1,0.297819,0.459570,0.574453,0.135099,0.027020,0.033775,0.142683,0.428543,...,0,0.258819,-9.659258e-01,0.433884,-0.900969,0.500000,-8.660254e-01,0.354605,-9.350162e-01,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65103,5,1,0.089551,0.285094,0.278123,0.039663,0.137933,0.140825,0.009099,0.133604,...,1,0.500000,8.660254e-01,0.433884,-0.900969,0.866025,5.000000e-01,0.822984,5.680647e-01,0
66723,5,1,0.530488,0.285105,0.907613,0.060874,0.137929,0.032043,0.113650,0.133602,...,1,0.500000,8.660254e-01,0.433884,-0.900969,0.866025,5.000000e-01,0.822984,5.680647e-01,0
26440,5,1,0.243512,0.285116,0.278141,0.253974,0.137924,0.140821,0.007153,0.133599,...,0,-1.000000,-1.836970e-16,0.433884,-0.900969,-0.866025,-5.000000e-01,-0.935016,-3.546049e-01,0
13073,5,1,0.530527,0.285133,0.529096,0.060868,0.137921,0.009864,0.113641,0.133595,...,1,0.500000,8.660254e-01,0.433884,-0.900969,0.866025,5.000000e-01,0.822984,5.680647e-01,3
